# 1. Setup and Imports

In [1]:
import sys
import os
import time
# Add the src directory to the path so we can import malthusjax
sys.path.append('/Users/leonardodicaterina/Documents/GitHub/MalthusJAX/src')

import jax
import jax.numpy as jnp
import jax.random as jar


print("JAX version:", jax.__version__)
print("Available devices:", jax.devices())

JAX version: 0.7.0
Available devices: [CpuDevice(id=0)]


# 1. BinaryGenome
The BinaryGenome class represents a binary string, which serves as the genetic material for our evolutionary algorithms.

In [2]:
from malthusjax.core.genome import BinaryGenome
# Create a valid genome
genome_init_params = {'array_size': 5, 'p': 0.5}

# Create a random genome
my_random_key = jar.PRNGKey(42)
random_genome = BinaryGenome(array_size=5, p=0.5, random_init=True, random_key=my_random_key)
print(f'Random genome:\n\t{random_genome}')

Random genome:
	[False False  True  True  True](size=5, valid=True)


## 1.1 Initialization
You can initialize a BinaryGenome by specifying custom ones like array_size and a probability p and either assigning a specific tensor or allowing manually or random initialization

In [3]:
from malthusjax.core.fitness.binary_ones import BinarySumFitnessEvaluator
from malthusjax.core.fitness.binary_ones import KnapsackFitnessEvaluator

evaluator_instance = BinarySumFitnessEvaluator()
binarysum_evaluator_fn = evaluator_instance.tensor_fitness_function

knapsack_weights = jnp.array([2, 3, 4, 5, 9])
knapsack_values = jnp.array([3, 4, 5, 8, 10])
knapsack_capacity = 20
knapsack_evaluator_fn = KnapsackFitnessEvaluator(
    weights=knapsack_weights,
    values=knapsack_values,
    weight_limit=knapsack_capacity
).tensor_fitness_function

fitness_random = binarysum_evaluator_fn(random_genome.to_tensor())
print(f'number of 1s in the random genome {random_genome.to_tensor()}: {fitness_random}')

fitness_knapsack = knapsack_evaluator_fn(random_genome.to_tensor())
print(f'Knapsack fitness of the random genome {random_genome.to_tensor()}: {fitness_knapsack}')

number of 1s in the random genome [0 0 1 1 1]: 3
Knapsack fitness of the random genome [0 0 1 1 1]: 23.0


you can transform a genome back to tensor


In [4]:
edge_case_genomes = [
    jnp.array([0, 0, 0, 0, 0]),  # All zeros
    jnp.array([1, 1, 1, 1, 1]),  # All ones
    jnp.array([1, 0, -1, 0, 1])  # Mixed values including invalid -1, I know this will give an error as the function is not designed for misrepresented genomes
]
edge_case_results = [3, 5, 0]



evaluator_instance.debug_tensor_fitness_function(edge_case_genomes, edge_case_results)

**************************************************
the tensor_fitness_function can be jit-compiled and vectorized correctly.
**************************************************



Evaluating edge cases:


--------------------
Error evaluating genome [0 0 0 0 0]: Expected 3, but got 0
--------------------
Genome [1 1 1 1 1] evaluated correctly with fitness 5.
--------------------
Error evaluating genome [ 1  0 -1  0  1]: Expected 0, but got 1


In [5]:
single_fn = evaluator_instance.get_tensor_fitness_function()
batch_fn = evaluator_instance.get_batch_fitness_function()
# show that it is a callable
print(f"single_fn is callable: {callable(single_fn)}")
print(f"batch_fn is callable: {callable(batch_fn)}")

single_fn is callable: True
batch_fn is callable: True


In [6]:
print(f"single genome evaluation: {evaluator_instance.evaluate_single(random_genome)}")
print(f"single genome evaluation: {evaluator_instance.evaluate_single(random_genome.to_tensor())}")

single genome evaluation: 3.0
single genome evaluation: 3.0


In [7]:
from malthusjax.core.population.base import AbstractPopulation
population = AbstractPopulation(
    genome_cls=BinaryGenome,
    pop_size=10,
    genome_init_params=genome_init_params,
    random_key=jar.PRNGKey(0),
    random_init=True
)

stack_of_genomes = population.to_stack()
batch_fitness = evaluator_instance.evaluate_batch(stack_of_genomes)
print(f"batch fitness from population: {batch_fitness}")
batch_fitness_v2 = batch_fn(stack_of_genomes)
print(f"batch fitness from population v2: {batch_fitness_v2}")
list_of_genomes = [population[i] for i in range(population._pop_size)]
list_fitness = evaluator_instance.evaluate_batch(list_of_genomes)
print(f"list fitness from population: {list_fitness}")


batch fitness from population: [4.0, 4.0, 2.0, 3.0, 4.0, 0.0, 1.0, 4.0, 4.0, 3.0]
batch fitness from population v2: [4 4 2 3 4 0 1 4 4 3]
list fitness from population: [4.0, 4.0, 2.0, 3.0, 4.0, 0.0, 1.0, 4.0, 4.0, 3.0]


# 2. CategoricalGenome
The CategoricalGenome class represents a categorical array, which serves as the genetic material for our evolutionary algorithms.

In [8]:
from malthusjax.core.genome.categorical import CategoricalGenome

# Create a valid genome
genome_init_params = {'array_size': 5, 'n_categories': 3}
# Create a random genome
my_random_key = jar.PRNGKey(42)
random_genome = CategoricalGenome(**genome_init_params, random_init=True, random_key=my_random_key)

## 2.1 Initialization
You can initialize a CategoricalGenome by specifying custom ones like array_size and a number of categories n_categories and either assigning a specific tensor or allowing manually or random initialization.

# 3. PermutationGenome
The PermutationGenome class represents a permutation array, which serves as the genetic material for our evolutionary algorithms.

In [9]:
from malthusjax.core.genome.permutation import PermutationGenome

genome_init_params = {'permutation_start': 0, 'permutation_end': 5}
# Create a random genome
my_random_key = jar.PRNGKey(42)
random_genome = PermutationGenome(**genome_init_params, random_init=True, random_key=my_random_key)
print(f'Random genome:\n\t{random_genome}')

Random genome:
	PermutationGenome(permutation_start=0, permutation_end=5 


## 3.1 Initialization
You can initialize a PermutationGenome by specifying custom parameters: permutation_start and permutation_end, and either assigning a specific tensor or allowing manually or random initialization.

In [10]:
from malthusjax.core.fitness.permutation import TSPFitnessEvaluator, SortingFitnessEvaluator

# Create a TSP fitness evaluator
distance_matrix = jnp.array([[0, 2, 9, 10, 7, 3],
                             [1, 0, 6, 4, 3, 8],
                             [15, 7, 0, 8, 12, 4],
                             [6, 3, 12, 0, 5, 2],
                             [10, 4, 8, 3, 0, 6],
                             [3, 8, 4, 2, 6, 0]])
tsp_evaluator = TSPFitnessEvaluator(distance_matrix)
tsp_evaluator_fn = tsp_evaluator.tensor_fitness_function
fitness_tsp = tsp_evaluator_fn(random_genome.to_tensor())
print(f'TSP fitness of the random genome {random_genome.to_tensor()}: {fitness_tsp}')

# Create a sorting fitness evaluator
sorting_evaluator = SortingFitnessEvaluator()
sorting_evaluator_fn = sorting_evaluator.tensor_fitness_function
fitness_sorting = sorting_evaluator_fn(random_genome.to_tensor())
print(f'Sorting fitness of the random genome {random_genome.to_tensor()}: {fitness_sorting}')

# penalty materix 6*6
penalty_matrix = jnp.array([[0, 1, 2, 3, 4, 5],
                            [1, 0, 1, 2, 3, 4],
                            [2, 1, 0, 1, 2, 3],
                            [3, 2, 1, 0, 1, 2],
                            [4, 3, 2, 1, 0, 1],
                            [5, 4, 3, 2, 1, 0]])
from malthusjax.core.fitness.permutation import FixedGroupingFitnessEvaluator
grouping_evaluator = FixedGroupingFitnessEvaluator(group_size=2, penalty_matrix=penalty_matrix)
grouping_evaluator_fn = grouping_evaluator.tensor_fitness_function
fitness_grouping = grouping_evaluator_fn(random_genome.to_tensor())
print(f'Grouping fitness of the random genome {random_genome.to_tensor()}: {fitness_grouping}')


TSP fitness of the random genome [2 0 4 3 1]: 28.0
Sorting fitness of the random genome [2 0 4 3 1]: -8.0
Grouping fitness of the random genome [2 0 4 3 1]: 6.0


## 3.2 Validation
when creating creating a genome from tensor it will automaticlly clip to 0 or (n_classes - 1) ensuring all elements are within boundaries, the only way to have an invalid genome is to manually assign a tensor with values outisde the specific range or assign a tensor of wrong size.

In [11]:
invalid_tensor_1 = jnp.array([0, 1, 1, 4, 3])
invalid_genome1 = PermutationGenome.from_tensor(invalid_tensor_1, genome_init_params= genome_init_params)
print(f'Invalid genome1 is valid: {invalid_genome1._validate()}')
invalid_tensor_2 = jnp.array([0, -1, 2, 3, 4])
invalid_genome2 = PermutationGenome.from_tensor(invalid_tensor_2, genome_init_params= genome_init_params)
print(f'Invalid genome2 is valid: {invalid_genome2._validate()}')
invalid_tensor_3 = jnp.arange(6)
try:
    invalid_genome3 = PermutationGenome.from_tensor(invalid_tensor_3, genome_init_params= genome_init_params)
    print(f'Invalid genome3 is valid: {invalid_genome3._validate()}')
except ValueError as e:
    print(f'Failed to create invalid genome3: {e}')


Genome contains duplicates: [0 1 1 4 3]
Genome contains duplicates: [0 1 1 4 3]
Invalid genome1 is valid: False
Genome values [ 0 -1  2  3  4] out of range [0, 5]
Genome values [ 0 -1  2  3  4] out of range [0, 5]
Invalid genome2 is valid: False
[0 1 2 3 4 5] = (5,)
[0 1 2 3 4 5] = (5,)
Invalid genome3 is valid: False


In [12]:
# Permutations are tricky to auto-correct, so we will skip that part

# 4 RealGenome
The RealGenome class represents a real-valued array, which serves as the genetic material for our evolutionary algorithms.

In [13]:
from malthusjax.core.genome.real import RealGenome

# Create a random genome
my_random_key = jar.PRNGKey(42)
genome_init_params = {'array_size': 5, 'minval': -1.0, 'maxval': 1.0}
random_genome = RealGenome(**genome_init_params, random_init=True, random_key=my_random_key)
print(f'Random genome:\n\t{random_genome}')

Random genome:
	RealGenome(size=5, valid=True)


## 4.1 Initialization
You can initialize a RealGenome by specifying custom ones like array_size and bounds (a tuple defining the minimum and maximum values) and either assigning a specific tensor or allowing manually or random initialization.

In [14]:
from malthusjax.core.fitness.real_ode import TaylorSeriesFitnessEvaluator
from malthusjax.core.fitness.real import SphereFitnessEvaluator, RastriginFitnessEvaluator, RosenbrockFitnessEvaluator, AckleyFitnessEvaluator, GriewankFitnessEvaluator

# Create a fitness evaluator
target_function = lambda x: jnp.sin(x)  # Example target function
x_values = jnp.linspace(0, jnp.pi, 100)  # Example x values


fitness_evaluator = TaylorSeriesFitnessEvaluator(
    target_function=target_function,
    x_values=x_values
)
taylor_fitness_fn = fitness_evaluator.tensor_fitness_function

fitness_random = taylor_fitness_fn(random_genome.to_tensor())
print(f'Taylor series fitness of the random genome {random_genome.to_tensor()}: {fitness_random}')

# Sphere fitness evaluator
sphere_evaluator = SphereFitnessEvaluator()
sphere_fitness_fn = sphere_evaluator.tensor_fitness_function
fitness_random_sphere = sphere_fitness_fn(random_genome.to_tensor())
print(f'Sphere fitness of the random genome {random_genome.to_tensor()}: {fitness_random_sphere}')


# Rastrigin fitness evaluator
rastrigin_evaluator = RastriginFitnessEvaluator()
rastrigin_fitness_fn = rastrigin_evaluator.tensor_fitness_function
fitness_random_rastrigin = rastrigin_fitness_fn(random_genome.to_tensor())
print(f'Rastrigin fitness of the random genome {random_genome.to_tensor()}: {fitness_random_rastrigin}')


# Rosenbrock fitness evaluator
rosenbrock_evaluator = RosenbrockFitnessEvaluator()
rosenbrock_fitness_fn = rosenbrock_evaluator.tensor_fitness_function
fitness_random_rosenbrock = rosenbrock_fitness_fn(random_genome.to_tensor())
print(f'Rosenbrock fitness of the random genome {random_genome.to_tensor()}: {fitness_random_rosenbrock}')

# Ackley fitness evaluator
ackley_evaluator = AckleyFitnessEvaluator()
ackley_fitness_fn = ackley_evaluator.tensor_fitness_function
fitness_random_ackley = ackley_fitness_fn(random_genome.to_tensor())
print(f'Ackley fitness of the random genome {random_genome.to_tensor()}: {fitness_random_ackley}')


# Griewank fitness evaluator
griewank_evaluator = GriewankFitnessEvaluator()
griewank_fitness_fn = griewank_evaluator.tensor_fitness_function
fitness_random_griewank = griewank_fitness_fn(random_genome.to_tensor())
print(f'Griewank fitness of the random genome {random_genome.to_tensor()}: {fitness_random_griewank}')


Evaluating fitness for genome tensor: [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]
Taylor series fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: -1097.77001953125
Sphere fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: 1.7755732536315918
Rastrigin fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: 84.90068054199219
Rosenbrock fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: 290.84881591796875
Ackley fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: 4.449814796447754
Griewank fitness of the random genome [ 0.45532846  0.5757351  -0.63661146 -0.47473955 -0.7785413 ]: 0.2972773313522339
